In [23]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

Overwriting mapper.py


In [2]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Writing reducer.py


In [30]:
%%writefile mapper2.py

import sys

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        word, count = line.strip().split('\t')
        count = int(count)
    except ValueError as e:
        continue
    print "%d\t%s" % (count, word)

Overwriting mapper2.py


In [31]:
%%writefile reducer2.py

#from __future__ import print_function
import sys

for line in sys.stdin:
    try:
        count, word = line.strip().split('\t')
        count = int(count)
    except ValueError as e:
        continue
    print "%s\t%d" % (word, count)

Overwriting reducer2.py


In [9]:
! hdfs dfs -ls /data/wiki

Found 1 items
drwxrwxrwx   - jovyan supergroup          0 2017-10-17 13:15 /data/wiki/en_articles_part


In [34]:
%%bash

OUT_DIR="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -combiner "python reducer.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

#hdfs dfs -cat ${OUT_DIR}/part-00000 | head

OUT_DIR2="wordcount_final_"$(date +"%s%6N")
NUM_REDUCERS=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR2} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
    -D mapred.text.key.comparator.options=-nr \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper2.py,reducer2.py \
    -mapper "python mapper2.py" \
    -reducer "python reducer2.py" \
    -input ${OUT_DIR} \
    -output ${OUT_DIR2} > /dev/null

hdfs dfs -cat ${OUT_DIR2}/part-00000 | head -7 | tail -1

is	126420


rm: `wordcount_result_1511822612672939': No such file or directory
17/11/27 22:43:36 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/11/27 22:43:36 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/11/27 22:43:36 INFO mapred.FileInputFormat: Total input files to process : 1
17/11/27 22:43:36 INFO mapreduce.JobSubmitter: number of splits:2
17/11/27 22:43:36 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1511817121749_0027
17/11/27 22:43:37 INFO impl.YarnClientImpl: Submitted application application_1511817121749_0027
17/11/27 22:43:37 INFO mapreduce.Job: The url to track the job: http://6a2156f454f2:8088/proxy/application_1511817121749_0027/
17/11/27 22:43:37 INFO mapreduce.Job: Running job: job_1511817121749_0027
17/11/27 22:43:43 INFO mapreduce.Job: Job job_1511817121749_0027 running in uber mode : false
17/11/27 22:43:43 INFO mapreduce.Job:  map 0% reduce 0%
17/11/27 22:43:59 INFO mapreduce.Job:  map 28% reduce 0%
17/11/27 22:4